In [3]:
import pickle

with open(f'networks/assignments/Graeme_PIPELINE_PYTHON_District_A_initial.pkl', 'rb') as file:
    network = pickle.load(file)

In [23]:
network['District_A']['assignments'].plot_interactive_assignments()

alt.LayerChart(...)